In [2]:
import pandas as pd

In [3]:
merged_census_rucc = pd.read_csv("../integration/merged_census_rucc.csv")
merged_census_rucc.head()


,State,Median_Income,Edu_Total,HS_Grad,Bachelors,Masters,Doctorate,state,Rural,Urban,Total,Urban_Pct,Rural_Pct,State_Type
0,Alabama,59674,3474924,874286,609316,286677,42908,1,36,31,67,46.268657,53.731343,Rural
1,Alaska,88121,489218,116008,94168,39551,6548,2,27,3,30,10.000000,90.000000,Rural
2,Arizona,74568,5053656,996330,1032052,463372,73242,4,7,8,15,53.333333,46.666667,Mixed
3,Arkansas,55432,2057624,583402,323961,144054,23987,5,58,17,75,22.666667,77.333333,Rural
4,California,91551,26866773,4846859,6056169,2643964,517699,6,21,37,58,63.793103,36.206897,Mixed


In [4]:
len(merged_census_rucc)

52

In [5]:
df_stroke = pd.read_csv("../integration/stroke_cleaned.csv")
df_stroke.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,1,67.0,0,1,1,Private,Urban,228.69,36.6,formerly smoked,1
1,31112,1,80.0,0,1,1,Private,Rural,105.92,32.5,never smoked,1
2,60182,0,49.0,0,0,1,Private,Urban,171.23,34.4,smokes,1
3,1665,0,79.0,1,0,1,Self-employed,Rural,174.12,24.0,never smoked,1
4,56669,1,81.0,0,0,1,Private,Urban,186.21,29.0,formerly smoked,1


In [6]:
len(df_stroke)

4909

#### Merging CSVs

In [7]:
import numpy as np

# Set seed for reproducibility
np.random.seed(42)


In [8]:
rural_states = merged_census_rucc[merged_census_rucc["State_Type"] == "Rural"]["state"].tolist()
urban_states = merged_census_rucc[merged_census_rucc["State_Type"] == "Urban"]["state"].tolist()
mixed_states = merged_census_rucc[merged_census_rucc["State_Type"] == "Mixed"]["state"].tolist()

In [9]:
def simulate_state(row):
    if row["Residence_type"] == "Rural":
        return np.random.choice(rural_states + mixed_states)
    elif row["Residence_type"] == "Urban":
        return np.random.choice(urban_states + mixed_states)
    else:
        return np.random.choice(mixed_states)

In [10]:
df_stroke["state"] = df_stroke.apply(simulate_state, axis=1)


In [11]:
df_stroke["state"] = df_stroke["state"].astype(str).str.zfill(2)
merged_census_rucc["state"] = merged_census_rucc["state"].astype(str).str.zfill(2)

In [12]:
final_df = pd.merge(df_stroke, merged_census_rucc, on="state", how="left")
final_df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,...,HS_Grad,Bachelors,Masters,Doctorate,Rural,Urban,Total,Urban_Pct,Rural_Pct,State_Type
0,9046,1,67.0,0,1,1,Private,Urban,228.69,36.6,...,568906,584999,345966,49857,2,7,9,77.777778,22.222222,Mixed
1,31112,1,80.0,0,1,1,Private,Rural,105.92,32.5,...,1259999,958365,391150,71956,51,44,95,46.315789,53.684211,Rural
2,60182,0,49.0,0,0,1,Private,Urban,171.23,34.4,...,852142,725469,345957,54737,20,26,46,56.521739,43.478261,Mixed
3,1665,0,79.0,1,0,1,Self-employed,Rural,174.12,24.0,...,840937,525044,218915,35332,27,37,64,57.812500,42.187500,Mixed
4,56669,1,81.0,0,0,1,Private,Urban,186.21,29.0,...,180292,151894,81198,14956,1,2,3,66.666667,33.333333,Mixed


In [13]:
final_df.to_csv("../integration/final_stroke_census_merged.csv", index=False)